In [ ]:

import pandas as pd
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv # اضافه شده
import torch.nn.functional as F # اضافه شده
from torch_geometric.nn import global_mean_pool # اضافه شده
from rdkit import Chem
from rdkit import RDLogger
from tqdm import tqdm

In [2]:


csv_path   = "pairs.csv"


RDLogger.DisableLog('rdApp.*')


def fix_smiles(smiles):
    return smiles.replace("N+[O-]", "[N+](=O)[O-]")


def atom_features(atom):
    symbol = atom.GetSymbol()
    one_hot = [
        1.0 if symbol == "H" else 0.0,
        1.0 if symbol == "C" else 0.0,
        1.0 if symbol == "N" else 0.0,
        1.0 if symbol == "O" else 0.0,
        1.0 if symbol == "F" else 0.0
    ]


    
    atomic_num = float(atom.GetAtomicNum())

    formal_charge = float(atom.GetFormalCharge())

    chirality_tag = float(int(atom.GetChiralTag()))

    is_aromatic = 1.0 if atom.GetIsAromatic() else 0.0
    is_in_ring  = 1.0 if atom.IsInRing() else 0.0

    degree_minus_1 = float(atom.GetDegree() - 1)

    feature_vector = (
        one_hot
        + [atomic_num]
        + [formal_charge, chirality_tag, is_aromatic, is_in_ring]
        + [degree_minus_1]
    )
    return torch.tensor(feature_vector, dtype=torch.float)


def mol_to_graph(smiles: str):
    smiles = fix_smiles(smiles)
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    try:
        
        x = torch.stack([atom_features(atom) for atom in mol.GetAtoms()])

        
        edge_index = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_index += [[i, j], [j, i]]

        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        if edge_index.numel() == 0:
            edge_index = torch.empty((2, 0), dtype=torch.long)

        return Data(x=x, edge_index=edge_index)

    except Exception:
        return None  


print("🔄 Loading CSV...")
df = pd.read_csv(csv_path)

graph_pairs = []
invalid_smiles = []

label2id = {
    "ADHD": 0,
    "Schizophrenia": 1,
    "Epilepsy": 2,
    "Supplement": 3
}

print("🔬 Processing molecules...")
for idx, row in tqdm(df.iterrows(), total=len(df)):
    smiles1 = row["mol_1"]
    smiles2 = row["mol_2"]
    label = label2id[row["label"]]  

    g1 = mol_to_graph(smiles1)
    g2 = mol_to_graph(smiles2)

    if (g1 is None) or (g2 is None):
        invalid_smiles.append((smiles1, smiles2))
        continue

    graph_pairs.append((g1, g2, label))

print(f"✅ loaded {len(graph_pairs)} graph pairs")


if invalid_smiles:
    print(f"⚠️ Skipped {len(invalid_smiles)} invalid SMILES pairs")
    with open("invalid_smiles.txt", "w") as f:
        for s1, s2 in invalid_smiles:
            f.write(f"{s1},{s2}\n")


🔄 Loading CSV...
🔬 Processing molecules...


100%|██████████| 14436/14436 [00:25<00:00, 563.84it/s]

✅ loaded 14156 graph pairs
⚠️ Skipped 280 invalid SMILES pairs


In [3]:

g1, g2, label = graph_pairs[0]

print("Molecule 1:", g1)
print("Molecule 2:", g2)
print("Label:", label)

print(g1.x)

Molecule 1: Data(x=[28, 11], edge_index=[2, 62])
Molecule 2: Data(x=[30, 11], edge_index=[2, 66])
Label: 1
tensor([[0., 1., 0., 0., 0., 6., 0., 0., 0., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 7., 0., 0., 0., 1., 2.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 0., 1., 2.],
        [0., 0., 1., 0., 0., 7., 0., 0., 1., 1., 2.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 2.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 2.],
        [0., 0., 1., 0., 0., 7., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 6., 0., 0., 1., 1., 2.],
        [0., 0., 0., 1., 0., 8., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 

In [4]:
import torch
import torch.nn as nn

class CrossAttention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        #change this
        self.query = nn.Linear(hidden_dim, hidden_dim)
        self.key   = nn.Linear(hidden_dim, hidden_dim)
        self.value = nn.Linear(hidden_dim, hidden_dim)
        self.scale = hidden_dim ** 0.5

    def forward(self, x_query, x_key_value):
        #change this
        Q = self.query(x_query)          
        K = self.key(x_key_value)         
        V = self.value(x_key_value)       

        attn_scores  = torch.matmul(Q, K.T) / self.scale   
        attn_weights = torch.softmax(attn_scores, dim=-1) 
        attended     = torch.matmul(attn_weights, V)      
        return attended

class CrossAttentionGNN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn1 = GCNConv(in_channels, hidden_channels)
        self.gcn2 = GCNConv(hidden_channels, hidden_channels)
        

        self.cross_attention = CrossAttention(hidden_channels)

        self.classifier = nn.Sequential(
            nn.Linear(2 * hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Linear(hidden_channels, 4)  
        )

    def forward(self, data_a, data_b):
        #change this
        #change this
        x1, edge_index1, batch1 = data_a.x, data_a.edge_index, data_a.batch
        x2, edge_index2, batch2 = data_b.x, data_b.edge_index, data_b.batch

        x1 = F.relu(self.gcn1(x1, edge_index1))
        x1 = F.relu(self.gcn2(x1, edge_index1))

        x2 = F.relu(self.gcn1(x2, edge_index2)) 
        x2 = F.relu(self.gcn2(x2, edge_index2))

        
        attn1 = self.cross_attention(x1, x2)  
        attn2 = self.cross_attention(x2, x1)

        
        pooled1 = global_mean_pool(attn1, batch1)
        pooled2 = global_mean_pool(attn2, batch2)

        
        combined = torch.cat([pooled1, pooled2], dim=1)

       
        out = self.classifier(combined)
        return out
    
    

In [5]:

import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from torch_geometric.data import Batch

def custom_collate(batch):
    data_a_list, data_b_list, labels = zip(*batch)
    batch_a = Batch.from_data_list(data_a_list)
    batch_b = Batch.from_data_list(data_b_list)
    labels = torch.tensor([int(l) if isinstance(l, str) else l for l in labels])  #
    return batch_a, batch_b, labels


train_size = int(0.8 * len(graph_pairs))
train_set, test_set = torch.utils.data.random_split(graph_pairs, [train_size, len(graph_pairs) - train_size])

train_loader = DataLoader(train_set, batch_size=16, shuffle=True, collate_fn=custom_collate)
test_loader = DataLoader(test_set, batch_size=16, collate_fn=custom_collate)


model = CrossAttentionGNN(in_channels=11, hidden_channels=100)  #change this
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#change this
criterion = torch.nn.CrossEntropyLoss() #change this


for epoch in range(1, 11): 
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    #complete this
    for batch in train_loader:
        data_a, data_b, labels = batch
        data_a = data_a.to(device)
        data_b = data_b.to(device)
        labels = torch.tensor(labels).to(device)

        optimizer.zero_grad()
        outputs = model(data_a, data_b)  

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * labels.size(0)

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)


    acc = correct / total
    print(f"Epoch {epoch:02d} | Train Loss: {total_loss:.4f} | Train Acc: {acc:.2%}")


model.eval()
all_preds = []
all_labels = []

torch.save(model.state_dict(), "trained_cross_attention_gnn.pth")
print("✅ مدل با موفقیت ذخیره شد.")

with torch.no_grad():
    #complete this
    for batch in test_loader:
        data_a, data_b, labels = batch
        data_a = data_a.to(device)
        data_b = data_b.to(device)
        labels = torch.tensor(labels).to(device)

        outputs = model(data_a, data_b)  
        preds = outputs.argmax(dim=1)   

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())

# === Classification Report ===
id2label = {0: "ADHD", 1: "Schizophrenia", 2: "Epilepsy", 3: "Supplement"}
print(classification_report(all_labels, all_preds, target_names=[id2label[i] for i in range(4)]))

C:\Users\IRANICA\AppData\Local\Temp\ipykernel_22736\2119641410.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


Epoch 01 | Train Loss: 13258.8658 | Train Acc: 52.70%
Epoch 02 | Train Loss: 10154.1737 | Train Acc: 59.02%
Epoch 03 | Train Loss: 9413.5295 | Train Acc: 61.60%
Epoch 04 | Train Loss: 8957.9294 | Train Acc: 63.74%
Epoch 05 | Train Loss: 6436.1289 | Train Acc: 76.75%
Epoch 06 | Train Loss: 4159.5178 | Train Acc: 85.07%
Epoch 07 | Train Loss: 3164.8158 | Train Acc: 89.07%
Epoch 08 | Train Loss: 2763.0171 | Train Acc: 90.45%
Epoch 09 | Train Loss: 2343.3876 | Train Acc: 91.98%
Epoch 10 | Train Loss: 2074.6182 | Train Acc: 93.22%
✅ مدل با موفقیت ذخیره شد.


C:\Users\IRANICA\AppData\Local\Temp\ipykernel_22736\2119641410.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels).to(device)


               precision    recall  f1-score   support

         ADHD       0.95      0.76      0.85       216
Schizophrenia       0.99      0.99      0.99       620
     Epilepsy       0.95      0.97      0.96      1436
   Supplement       0.93      0.95      0.94       560

     accuracy                           0.95      2832
    macro avg       0.95      0.92      0.93      2832
 weighted avg       0.95      0.95      0.95      2832

